In [5]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.6 MB/s eta 0:00:0000:0100:01


In [3]:
from datasets import load_dataset
import tokenizers
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, pipeline, Trainer, TrainingArguments

In [4]:
text = """This isn't my typical movie genre. I am more interested in character drama than action movies. This is able to span both genres in a way that [MASK] hasn't managed to accomplish. The plot was interesting and the acting by Anne Parillaud was exceptional.<br /><br />I had seen the American remake, called "Point of No Return" and found it to be an OK movie, but I wanted to see the original. I am glad that I did. The technical aspects of direction and editing are very good. The plot was interesting - revolving around the timeless theme of redemption. The acting made this an exceptional movie.<br /><br />For fans of the action genre, they will be quite happy. For those like myself who are more interested in character and plot-driven movies may find this to be a pleasant surprise as well."""

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [6]:
dataset = load_dataset("imdb")
test_dataset = dataset["test"].map(lambda e: tokenizer(e["text"], truncation=True, padding=True), batched=True)
test_dataset = test_dataset.rename_column("label", "labels")

# Using HF BERT OG

In [11]:
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
fill_mask = pipeline(
    "fill-mask",
    model=bert_model,
    tokenizer=tokenizer
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [12]:
fill_mask(text)

[{'score': 0.737318217754364,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'this isn\'t my typical movie genre. i am more interested in character drama than action movies. this is able to span both genres in a way that it hasn\'t managed to accomplish. the plot was interesting and the acting by anne parillaud was exceptional. < br / > < br / > i had seen the american remake, called " point of no return " and found it to be an ok movie, but i wanted to see the original. i am glad that i did. the technical aspects of direction and editing are very good. the plot was interesting - revolving around the timeless theme of redemption. the acting made this an exceptional movie. < br / > < br / > for fans of the action genre, they will be quite happy. for those like myself who are more interested in character and plot - driven movies may find this to be a pleasant surprise as well.'},
 {'score': 0.04811081290245056,
  'token': 2028,
  'token_str': 'one',
  'sequence': 'this isn\'t my typ

In [13]:
training_args = TrainingArguments(
    per_device_train_batch_size=32,
)

trainer = Trainer(
    model=bert_model,
    args=training_args,
    data_collator=data_collator,
)

trainer.evaluate(test_dataset)

{'eval_loss': 2.345867872238159,
 'eval_model_preparation_time': 0.0022,
 'eval_runtime': 317.0183,
 'eval_samples_per_second': 78.86,
 'eval_steps_per_second': 9.857}

# Using Wes BERT

Wes BERT 

In [18]:
model = BertForMaskedLM.from_pretrained('checkpoints/checkpoint-95000')
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [19]:
fill_mask(text)

[{'score': 0.29947608709335327,
  'token': 5365,
  'token_str': 'hollywood',
  'sequence': 'this isn\'t my typical movie genre. i am more interested in character drama than action movies. this is able to span both genres in a way that hollywood hasn\'t managed to accomplish. the plot was interesting and the acting by anne parillaud was exceptional. < br / > < br / > i had seen the american remake, called " point of no return " and found it to be an ok movie, but i wanted to see the original. i am glad that i did. the technical aspects of direction and editing are very good. the plot was interesting - revolving around the timeless theme of redemption. the acting made this an exceptional movie. < br / > < br / > for fans of the action genre, they will be quite happy. for those like myself who are more interested in character and plot - driven movies may find this to be a pleasant surprise as well.'},
 {'score': 0.296558678150177,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'this 

In [20]:
training_args = TrainingArguments(
    per_device_train_batch_size=32,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
)

In [21]:
trainer.evaluate(test_dataset)

{'eval_loss': 2.4548254013061523,
 'eval_model_preparation_time': 0.0012,
 'eval_runtime': 227.6852,
 'eval_samples_per_second': 109.801,
 'eval_steps_per_second': 13.725}

# Using HF BERT PRO

In [8]:
model = BertForMaskedLM.from_pretrained(
    "HamsterShiu/BERT_MLM",
    subfolder="hf_bert_pro_20_epochs",
)
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [9]:
fill_mask(text)

[{'score': 0.6361707448959351,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'this isn\'t my typical movie genre. i am more interested in character drama than action movies. this is able to span both genres in a way that it hasn\'t managed to accomplish. the plot was interesting and the acting by anne parillaud was exceptional. < br / > < br / > i had seen the american remake, called " point of no return " and found it to be an ok movie, but i wanted to see the original. i am glad that i did. the technical aspects of direction and editing are very good. the plot was interesting - revolving around the timeless theme of redemption. the acting made this an exceptional movie. < br / > < br / > for fans of the action genre, they will be quite happy. for those like myself who are more interested in character and plot - driven movies may find this to be a pleasant surprise as well.'},
 {'score': 0.26585736870765686,
  'token': 5365,
  'token_str': 'hollywood',
  'sequence': 'this isn\'t

In [10]:
training_args = TrainingArguments(
    per_device_train_batch_size=32,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
)

trainer.evaluate(test_dataset)

{'eval_loss': 1.6976091861724854,
 'eval_model_preparation_time': 0.0018,
 'eval_runtime': 318.4947,
 'eval_samples_per_second': 78.494,
 'eval_steps_per_second': 9.812}